In [1]:
import warnings
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray as rxr
import geopandas as gpd
import rasterio
from rasterio.warp import transform_bounds
from rasterio.windows import from_bounds
from pyproj import Transformer, CRS
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import pystac_client
import fiona
import rtree
import planetary_computer
from odc.stac import stac_load

# Suppress warnings
warnings.filterwarnings("ignore")

In [2]:
import geopandas as gpd
import pandas as pd
import numpy as np
import fiona
import rtree
from tqdm import tqdm

def add_building_features_fast(gdf, building_footprints_path):
    """Calculates building features, including summary statistics."""
    gdf = gdf.to_crs("EPSG:32618")  # Convert to UTM for buffering and area calculation

    # --- Load Building Footprints (KML) ---
    try:
        layers = fiona.listlayers(building_footprints_path)
        print("Layers in KML:", layers)
        layer_name = layers[0]
    except Exception as e:
        print(f"Error listing KML layers: {e}")
        return None

    try:
        buildings_gdf = gpd.read_file(
            building_footprints_path,
            driver="KML",
            layer=layer_name,
        )
    except Exception as e:
        print(f"Error reading KML: {e}")
        return None

    if buildings_gdf.empty:
        print("No building footprints found in KML.")
        gdf = gdf.copy()
        gdf[['building_density', 'num_buildings', 'avg_building_area', 'max_building_area',
             'std_building_area', 'building_coverage', 'avg_perimeter_area_ratio',
             'std_perimeter_area_ratio', 'avg_building_height', 'max_building_height',
             'total_building_volume', 'volume_density', 'avg_compactness']] = 0
        return gdf

    # Ensure buildings are in UTM
    buildings_gdf = buildings_gdf.to_crs("EPSG:32618")

    # --- Create Spatial Index (R-tree) ---
    spatial_index = rtree.index.Index()
    for idx, building in enumerate(buildings_gdf.geometry):
        spatial_index.insert(idx, building.bounds)

    # --- Process Each Point ---
    def process_point(point):
        buffer = point.buffer(200)
        possible_matches_idx = list(spatial_index.intersection(buffer.bounds))
        possible_matches = buildings_gdf.iloc[possible_matches_idx]
        intersecting_buildings = possible_matches[possible_matches.geometry.intersects(buffer)]

        if not intersecting_buildings.empty:
            areas = intersecting_buildings.geometry.area
            perimeters = intersecting_buildings.geometry.length
            density = areas.sum() / buffer.area
            num_buildings = len(intersecting_buildings)
            avg_building_area = areas.mean()
            max_building_area = areas.max()
            std_building_area = areas.std()
            building_coverage = areas.sum()
            avg_perimeter_area_ratio = (perimeters / areas).mean()
            std_perimeter_area_ratio = (perimeters / areas).std()
            avg_compactness = ((4 * np.pi * areas) / (perimeters ** 2)).mean()

            if 'height' in intersecting_buildings.columns:
                heights = pd.to_numeric(intersecting_buildings['height'], errors='coerce')
                avg_building_height = heights.mean()
                max_building_height = heights.max()
                total_building_volume = (areas * heights).sum()
            else:
                avg_building_height = np.nan
                max_building_height = np.nan
                total_building_volume = np.nan

            volume_density = total_building_volume / buffer.area if total_building_volume else 0.0
        else:
            density = num_buildings = avg_building_area = max_building_area = 0.0
            std_building_area = building_coverage = avg_perimeter_area_ratio = 0.0
            std_perimeter_area_ratio = volume_density = avg_compactness = 0.0
            avg_building_height = max_building_height = total_building_volume = np.nan

        return pd.Series({
            'building_density': density,
            'num_buildings': num_buildings,
            'avg_building_area': avg_building_area,
            'max_building_area': max_building_area,
            'std_building_area': std_building_area,
            'building_coverage': building_coverage,
            'avg_perimeter_area_ratio': avg_perimeter_area_ratio,
            'std_perimeter_area_ratio': std_perimeter_area_ratio,
            'avg_building_height': avg_building_height,
            'max_building_height': max_building_height,
            'total_building_volume': total_building_volume,
            'volume_density': volume_density,
            'avg_compactness': avg_compactness
        })

    # Apply function to each point
    results = gdf.geometry.apply(process_point)

    # Add results to GeoDataFrame
    gdf = gdf.copy()
    gdf = pd.concat([gdf, results], axis=1)

    gdf = gdf.to_crs("EPSG:4326")  # Convert back to WGS 84
    return gdf

In [24]:
def median_mosaic_generation():
    # Define the bounding box
    lower_left = (40.75, -74.01)
    upper_right = (40.88, -73.86)
    bounds = (lower_left[1], lower_left[0], upper_right[1], upper_right[0])

    # Define the time window
    time_window = "2022-01-01/2024-12-31"

    # Search the Planetary Computer's STAC endpoint
    stac = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
    search = stac.search(
        bbox=bounds,
        datetime=time_window,
        collections=["landsat-c2-l2"],
        query={"eo:cloud_cover": {"lt": 40}, "platform": {"in": ["landsat-8"]}},
    )
    items = list(search.get_items())
    print('Number of scenes found:', len(items))

    # Define pixel resolution and scale
    resolution = 5  # meters per pixel
    scale = resolution / 111320.0  # degrees per pixel for crs=4326, NOT USED with UTM

    # Load data using stac_load (separate RGB/NIR and LST)
    #  Reproject to UTM during load
    data1 = stac_load(
        items,
        bands=["red", "green", "blue", "nir08",'swir16','swir22'],
        crs="EPSG:32618",  # UTM Zone 18N
        resolution=resolution, # Use resolution directly
        chunks={"x": 2048, "y": 2048},
        dtype="uint16",
        patch_url=planetary_computer.sign,
        bbox=bounds
    )
    data2 = stac_load(
        items,
        bands=["lwir11"],
        crs="EPSG:32618",  # UTM Zone 18N
        resolution=resolution, # Use resolution directly
        chunks={"x": 2048, "y": 2048},
        dtype="uint16",
        patch_url=planetary_computer.sign,
        bbox=bounds
    )

    # Persist data in memory
    data1 = data1.persist()
    data2 = data2.persist()

    # --- Scaling Datasets ---

    # Scale Factors for the RGB and NIR bands
    scale1 = 0.0000275
    offset1 = -0.2
    data1 = data1.astype(float) * scale1 + offset1

    # Scale Factors for the Surface Temperature band (Kelvin to Celsius)
    scale2 = 0.00341802
    offset2 = 149.0
    kelvin_celsius = 273.15
    data2 = data2.astype(float) * scale2 + offset2 - kelvin_celsius

    # --- Median Mosaic Generation ---
    data1_median = data1.median(dim="time")
    data2_median = data2.median(dim="time")

    # --- Save the output data ---
    filename = "Landsat_LST_Median5.tiff"
    height = data2_median.dims["y"]  # Use 'y' and 'x' for UTM
    width = data2_median.dims["x"]
    # Get transform directly from xarray object
    transform = data2_median.rio.transform()
    data2_median.rio.write_crs("EPSG:32618", inplace=True) # UTM Zone 18N
    data2_median.rio.write_transform(transform=transform, inplace=True)

    with rasterio.open(filename, 'w', driver='GTiff', width=width, height=height,
                      crs="EPSG:32618", transform=transform, count=7, compress='lzw', dtype='float64',num_threads="all_cpus") as dst:
        dst.write(data2_median.lwir11, 1)
        dst.write(data1_median.nir08, 2)
        dst.write(data1_median.red, 3)
        dst.write(data1_median.blue, 4)
        dst.write(data1_median.green, 5)
        dst.write(data1_median.swir16, 6)
        dst.write(data1_median.swir22, 7)
        dst.close()
    print(f"Saved median mosaic to {filename}")

median_mosaic_generation()

Number of scenes found: 91
Saved median mosaic to Landsat_LST_Median5.tiff


In [52]:
def sentinel_median():
    # Define the bounding box and time window
    lower_left = (40.75, -74.01)
    upper_right = (40.88, -73.86)
    bounds = (lower_left[1], lower_left[0], upper_right[1], upper_right[0])
    time_window = "2022-01-01/2024-12-31"

    # Open STAC client
    stac = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")

    # Search for Sentinel-2 L2A data
    search = stac.search(
        bbox=bounds,
        datetime=time_window,
        collections=["sentinel-2-l2a"],
        query={"eo:cloud_cover": {"lt": 40}},
    )
    items = list(search.get_items())
    print('Number of scenes found:', len(items))

    # Define scale
    resolution = 10  # meters
    scale = resolution / 111320.0  # Not used when using UTM

    # Define the bands to load (only B01, B04, B06, B08)
    bands = ["B01", "B02", "B03", "B04", "B05", "B06", "B07", "B08","B8A","B11","B12"]

    # Load data, reprojecting to UTM Zone 18N
    data = stac_load(
        items,
        bands=bands,
        crs="EPSG:32618",  # UTM Zone 18N
        resolution=resolution,  # Use resolution directly
        chunks={"x": 2048, "y": 2048},
        dtype="uint16",
        patch_url=planetary_computer.sign,
        bbox=bounds,
    )
    data = data.persist()
    # Calculate median mosaic
    median_data = data.median(dim="time")

    # --- Save to GeoTIFF ---
    filename = "S2_median_composite5.tif"
    height = median_data.dims["y"]  # Use 'y' and 'x' for UTM
    width = median_data.dims["x"]
    # Get the transform directly from the xarray object
    transform = median_data.rio.transform()
    median_data.rio.write_crs("EPSG:32618", inplace=True) # UTM Zone 18N
    median_data.rio.write_transform(transform=transform, inplace=True)

    # Get the dtype from one of the DataArrays (e.g., B01)
    first_band = bands[0]  # Get the name of the first band
    data_dtype = median_data[first_band].dtype  # Get the dtype

    with rasterio.open(filename, 'w', driver='GTiff', width=width, height=height,
                       crs="EPSG:32618", transform=transform, count=len(bands), compress='lzw',
                       dtype=data_dtype,num_threads="all_cpus") as dst:  # Use the retrieved dtype
        for i, band in enumerate(bands):
            dst.write(median_data[band].values, i + 1)
    print(f"Saved Sentinel-2 median mosaic to {filename}")

sentinel_median()

Number of scenes found: 203
Saved Sentinel-2 median mosaic to S2_median_composite5.tif


In [3]:
def extract_buffered_values(tiff_path, csv_path, buffer_size_meters, bands=None):

    with rasterio.open(tiff_path) as src:
        transform = src.transform
        src_crs = src.crs

        if bands is None:
            bands = list(range(1, src.count + 1))

        df = pd.read_csv(csv_path)
        gdf = gpd.GeoDataFrame(
            df,
            geometry=gpd.points_from_xy(df.Longitude, df.Latitude),
            crs="EPSG:4326",  # WGS 84
        )

        gdf_proj = gdf.to_crs("EPSG:32618")
        gdf_proj['geometry'] = gdf_proj.geometry.buffer(buffer_size_meters)
        gdf = gdf_proj.to_crs(src_crs)

        band_data = {band: [] for band in bands}

        for point in tqdm(gdf.geometry, desc="Extracting Buffered Values"):
            minx, miny, maxx, maxy = point.bounds
            window = from_bounds(minx, miny, maxx, maxy, transform)

            try:
                for band in bands:
                    window_data = src.read(band, window=window, masked=True)
                    mean_value = np.nanmean(window_data)
                    band_data[band].append(mean_value)
            except rasterio.RasterioIOError as e:
                print(f"Error reading window: {e}")
                for band in bands:
                    band_data[band].append(np.nan)

        result_df = pd.DataFrame(band_data)
        return result_df

In [4]:
ground_df = pd.read_csv("Training_data_uhi_index.csv")

# Extract buffered Sentinel-2 values (B01, B04, B06, B08)
buffer_size = 150  # meters
sentinel_bands = [1, 2, 3, 4,5,6,7,8,9,10,11]
sentinel_buffered = extract_buffered_values("S2_median_composite5.tif", "Training_data_uhi_index.csv", buffer_size, sentinel_bands)
sentinel_buffered.rename(columns={
    1: 'B01',
    2: 'B02',
    3: 'B03',
    4: 'B04',
    5: 'B05',
    6: 'B06',
    7: 'B07',
    8: 'B08',
    9: 'B8A',
    10: 'B11',
    11: 'B12'
}, inplace=True)

# Extract buffered Landsat values (lst, nir08, red)
landsat_bands = [1, 2, 3,4,5,6,7]
landsat_buffered = extract_buffered_values("Landsat_LST_Median5.tiff", "Training_data_uhi_index.csv", buffer_size, landsat_bands)
landsat_buffered.rename(columns={
    1: 'lwir11',
    2: 'nir08',
    3: 'red',
    4: 'blue',
    5: 'green',
    6:'swir16',
    7:'swir22'
}, inplace=True)

# Combine dataframes
ground_df = pd.concat([ground_df, sentinel_buffered, landsat_buffered], axis=1)

# Calculate NDVI
ground_df['NDVI'] = (ground_df['B08'] - ground_df['B04']) / (ground_df['B08'] + ground_df['B04'])
ground_df['NDVI'] = ground_df['NDVI'].replace([np.inf, -np.inf], np.nan)

ground_df['NDVI2'] = (ground_df['nir08'] - ground_df['red']) / (ground_df['nir08'] + ground_df['red'])
ground_df['NDVI2'] = ground_df['NDVI2'].replace([np.inf, -np.inf], np.nan)

ground_df['NDBI'] = (ground_df['B11'] - ground_df['B08']) / (ground_df['B11'] + ground_df['B08'])
ground_df['NDBI'] = ground_df['NDBI'].replace([np.inf, -np.inf], np.nan)

ground_df['NDBI2'] = (ground_df['swir16'] - ground_df['nir08']) / (ground_df['swir16'] + ground_df['nir08'])
ground_df['NDBI2'] = ground_df['NDBI2'].where(~ground_df['NDBI2'].isnull(), other=np.nan)

ground_df['MNDWI'] = (ground_df['B03'] - ground_df['B11']) / (ground_df['B03'] + ground_df['B11'])
ground_df['MNDWI'] = ground_df['MNDWI'].where(~ground_df['MNDWI'].isnull(), other=np.nan)

ground_df['MNDWI2'] = (ground_df['green'] - ground_df['swir16']) / (ground_df['swir16'] + ground_df['green'])
ground_df['MNDWI2'] = ground_df['MNDWI2'].replace([np.inf, -np.inf], np.nan)

ground_df['EVI'] = 2.5 * (ground_df['B08'] - ground_df['B04']) / (ground_df['B08'] + 6 * ground_df['B04'] - 7.5 * ground_df['B02'] + 1)
ground_df['EVI'] = ground_df['EVI'].where(~ground_df['EVI'].isnull(), other=np.nan)

ground_df['EVI2'] = 2.5 * (ground_df['nir08'] - ground_df['red']) / (ground_df['nir08'] + 6 * ground_df['red'] - 7.5 * ground_df['blue'] + 1)
ground_df['EVI2'] = ground_df['EVI2'].where(~ground_df['EVI2'].isnull(), other=np.nan)

L = 0.5  # Common default value for L
ground_df['SAVI'] = ((ground_df['B08'] - ground_df['B04']) / (ground_df['B08'] + ground_df['B04'] + L)) * (1 + L)
ground_df['SAVI'] = ground_df['SAVI'].where(~ground_df['SAVI'].isnull(), other=np.nan)

ground_df['SAVI2'] = ((ground_df['nir08'] - ground_df['red']) / (ground_df['nir08'] + ground_df['red'] + L)) * (1 + L)
ground_df['SAVI2'] = ground_df['SAVI2'].where(~ground_df['SAVI2'].isnull(), other=np.nan)

ground_df['NDBaI'] = (ground_df['swir16'] - ground_df['lwir11']) / (ground_df['swir16'] + ground_df['lwir11'])
ground_df['NDBaI'] = ground_df['NDBaI'].where(~ground_df['NDBaI'].isnull(), other=np.nan)

ground_df['UI'] = (ground_df['B12'] - ground_df['B08']) / (ground_df['B12'] + ground_df['B08'])
ground_df['UI'] = ground_df['UI'].where(~ground_df['UI'].isnull(), other=np.nan)

ground_df['UI2'] = (ground_df['swir22'] - ground_df['nir08']) / (ground_df['swir22'] + ground_df['nir08'])
ground_df['UI2'] = ground_df['UI2'].where(~ground_df['UI2'].isnull(), other=np.nan)

ground_df['IBI'] = (ground_df['NDBI']-(ground_df['SAVI']+ground_df['MNDWI'])/2)/(ground_df['NDBI']+(ground_df['SAVI']+ground_df['MNDWI'])/2)
ground_df['IBI'] = ground_df['IBI'].where(~ground_df['IBI'].isnull(), other=np.nan)

ground_df['IBI2'] = (ground_df['NDBI2']-(ground_df['SAVI2']+ground_df['MNDWI2'])/2)/(ground_df['NDBI2']+(ground_df['SAVI2']+ground_df['MNDWI2'])/2)
ground_df['IBI2'] = ground_df['IBI2'].where(~ground_df['IBI2'].isnull(), other=np.nan)

ground_df['TVDI2'] = (ground_df['lwir11'] / 10 - ground_df['NDVI2']) / (ground_df['lwir11'] / 10)
ground_df['TVDI2'] = ground_df['TVDI2'].replace([np.inf, -np.inf], np.nan) #handle nan

ground_df['VANDVI'] = (1 - ground_df['NDVI']) * ground_df['NDBI']
ground_df['VANDVI'] = ground_df['VANDVI'].replace([np.inf, -np.inf], np.nan) #handle nan

ground_df['VANDVI2'] = (1 - ground_df['NDVI2']) * ground_df['NDBI2']
ground_df['VANDVI2'] = ground_df['VANDVI2'].replace([np.inf, -np.inf], np.nan) #handle nan

ground_df['LST_Minus_NDVI'] = (ground_df['lwir11'] / 10) - ground_df['NDVI2']
ground_df['LST_Minus_NDVI'] = ground_df['LST_Minus_NDVI'].replace([np.inf, -np.inf], np.nan)

Extracting Buffered Values: 100%|██████████| 11229/11229 [00:23<00:00, 486.31it/s]


In [5]:
ground_df = gpd.GeoDataFrame(
    ground_df,
    geometry=gpd.points_from_xy(ground_df.Longitude, ground_df.Latitude),
    crs="EPSG:4326",
)
ground_df = add_building_features_fast(ground_df, "Building_Footprint.kml")

Layers in KML: ['Challenge_footprint']


In [6]:
ground_df['NDBI_bd1'] = ground_df['NDBI'] * ground_df['building_density']
ground_df['NDBI_bd1'] = ground_df['NDBI_bd1'].replace([np.inf, -np.inf], np.nan) #handle nan

ground_df['NDBI_bd2'] = ground_df['NDBI2'] * ground_df['building_density']
ground_df['NDBI_bd2'] = ground_df['NDBI_bd2'].replace([np.inf, -np.inf], np.nan) #handle nan

In [7]:
from sklearn.feature_selection import RFECV

import xgboost as xgb
# --- Prepare Data for ML Model ---
ground_df = ground_df.drop_duplicates()
features = ['lwir11','B01', 'B06', 'NDVI2','building_coverage', 'avg_perimeter_area_ratio', 'std_building_area',
             'avg_building_area', 'max_building_area','B11','EVI','LST_Minus_NDVI','IBI2','NDBaI','MNDWI','SAVI','MNDWI2',
             'avg_compactness']

X = ground_df[features]
y = ground_df["UHI Index"]

# Handle missing values
X = X.fillna(X.mean())
y = y.fillna(y.mean())

# Scale Features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.05, random_state=42)

# --- Train and Evaluate Model ---
rf_model = RandomForestRegressor(n_estimators=500, random_state=42,n_jobs=-1, min_samples_split= 2, 
                                 min_samples_leaf= 1, max_features='sqrt', max_depth= None, bootstrap= False)
rf_model.fit(X_train, y_train)

y_train_pred = rf_model.predict(X_train)
y_test_pred = rf_model.predict(X_test)

print("In-Sample R² Score:", r2_score(y_train, y_train_pred))
print("Out-of-Sample R² Score:", r2_score(y_test, y_test_pred))

In-Sample R² Score: 0.9997648246772389
Out-of-Sample R² Score: 0.9742205622511698


In [86]:
 #--- Prediction on Submission Data ---
submission_df = pd.read_csv("Submission_template.csv")

# Extract buffered Sentinel and Landsat values (submission)
sentinel_buffered_submission = extract_buffered_values("S2_median_composite5.tif", "Submission_template.csv", buffer_size, sentinel_bands)
sentinel_buffered_submission.rename(columns={1: 'B01', 2: 'B02', 3: 'B03', 4: 'B04',5:"B05",6:"B06",7:"B07",8:"B08",9:"B8A",10:"B11",11:"B12"}, inplace=True)

landsat_buffered_submission = extract_buffered_values("Landsat_LST_Median5.tiff", "Submission_template.csv", buffer_size, landsat_bands)
landsat_buffered_submission.rename(columns={1: 'lwir11', 2: 'nir08', 3: 'red',4:'blue',5:'green',6:'swir16',7:'swir22'}, inplace=True)

# Combine submission data
submission_df = pd.concat([submission_df, sentinel_buffered_submission, landsat_buffered_submission], axis=1)

# Calculate NDVI for submission data
submission_df['NDVI'] = (submission_df['B08'] - submission_df['B04']) / (submission_df['B08'] + submission_df['B04'])
submission_df['NDVI'] = submission_df['NDVI'].replace([np.inf, -np.inf], np.nan)


submission_df['NDVI2'] = (submission_df['nir08'] - submission_df['red']) / (submission_df['nir08'] + submission_df['red'])
submission_df['NDVI2'] = submission_df['NDVI2'].replace([np.inf, -np.inf], np.nan)


submission_df['NDBI'] = (submission_df['B11'] - submission_df['B08']) / (submission_df['B11'] + submission_df['B08'])
submission_df['NDBI'] = submission_df['NDBI'].replace([np.inf, -np.inf], np.nan)


submission_df['NDBI2'] = (submission_df['swir16'] - submission_df['nir08']) / (submission_df['swir16'] + submission_df['nir08'])
submission_df['NDBI2'] = submission_df['NDBI2'].where(~submission_df['NDBI2'].isnull(), other=np.nan)


submission_df['MNDWI'] = (submission_df['B03'] - submission_df['B11']) / (submission_df['B03'] + submission_df['B11'])
submission_df['MNDWI'] = submission_df['MNDWI'].where(~submission_df['MNDWI'].isnull(), other=np.nan)


submission_df['MNDWI2'] = (submission_df['green'] - submission_df['swir16']) / (submission_df['swir16'] + submission_df['green'])
submission_df['MNDWI2'] = submission_df['MNDWI2'].replace([np.inf, -np.inf], np.nan)


submission_df['EVI'] = 2.5 * (submission_df['B08'] - submission_df['B04']) / (submission_df['B08'] + 6 * submission_df['B04'] - 7.5 * submission_df['B02'] + 1)
submission_df['EVI'] = submission_df['EVI'].where(~submission_df['EVI'].isnull(), other=np.nan)


submission_df['EVI2'] = 2.5 * (submission_df['nir08'] - submission_df['red']) / (submission_df['nir08'] + 6 * submission_df['red'] - 7.5 * submission_df['blue'] + 1)
submission_df['EVI2'] = submission_df['EVI2'].where(~submission_df['EVI2'].isnull(), other=np.nan)


L = 0.5  # Common default value for L
submission_df['SAVI'] = ((submission_df['B08'] - submission_df['B04']) / (submission_df['B08'] + submission_df['B04'] + L)) * (1 + L)
submission_df['SAVI'] = submission_df['SAVI'].where(~submission_df['SAVI'].isnull(), other=np.nan)


submission_df['SAVI2'] = ((submission_df['nir08'] - submission_df['red']) / (submission_df['nir08'] + submission_df['red'] + L)) * (1 + L)
submission_df['SAVI2'] = submission_df['SAVI2'].where(~submission_df['SAVI2'].isnull(), other=np.nan)


submission_df['NDBaI'] = (submission_df['swir16'] - submission_df['lwir11']) / (submission_df['swir16'] + submission_df['lwir11'])
submission_df['NDBaI'] = submission_df['NDBaI'].where(~submission_df['NDBaI'].isnull(), other=np.nan)


submission_df['UI'] = (submission_df['B12'] - submission_df['B08']) / (submission_df['B12'] + submission_df['B08'])
submission_df['UI'] = submission_df['UI'].where(~submission_df['UI'].isnull(), other=np.nan)


submission_df['UI2'] = (submission_df['swir22'] - submission_df['nir08']) / (submission_df['swir22'] + submission_df['nir08'])
submission_df['UI2'] = submission_df['UI2'].where(~submission_df['UI2'].isnull(), other=np.nan)


submission_df['IBI'] = (submission_df['NDBI']-(submission_df['SAVI']+submission_df['MNDWI'])/2)/(submission_df['NDBI']+(submission_df['SAVI']+submission_df['MNDWI'])/2)
submission_df['IBI'] = submission_df['IBI'].where(~submission_df['IBI'].isnull(), other=np.nan)


submission_df['IBI2'] = (submission_df['NDBI2']-(submission_df['SAVI2']+submission_df['MNDWI2'])/2)/(submission_df['NDBI2']+(submission_df['SAVI2']+submission_df['MNDWI2'])/2)
submission_df['IBI2'] = submission_df['IBI2'].where(~submission_df['IBI2'].isnull(), other=np.nan)

submission_df['TVDI2'] = (submission_df['lwir11'] / 10 - submission_df['NDVI2']) / (submission_df['lwir11'] / 10)
submission_df['TVDI2'] = submission_df['TVDI2'].replace([np.inf, -np.inf], np.nan) #handle nan

submission_df = gpd.GeoDataFrame(
    submission_df,
    geometry=gpd.points_from_xy(submission_df.Longitude, submission_df.Latitude),
    crs="EPSG:4326",
)
submission_df = add_building_features_fast(submission_df, "Building_Footprint.kml")
submission_df['NDBI_bd1'] = submission_df['NDBI'] * submission_df['building_density']
submission_df['NDBI_bd1'] = submission_df['NDBI_bd1'].replace([np.inf, -np.inf], np.nan) #handle nan

submission_df['NDBI_bd2'] = submission_df['NDBI2'] * submission_df['building_density']
submission_df['NDBI_bd2'] = submission_df['NDBI_bd2'].replace([np.inf, -np.inf], np.nan) #handle nan

submission_df['VANDVI'] = (1 - submission_df['NDVI']) * submission_df['NDBI']
submission_df['VANDVI'] = submission_df['VANDVI'].replace([np.inf, -np.inf], np.nan) #handle nan


submission_df['VANDVI2'] = (1 - submission_df['NDVI2']) * submission_df['NDBI2']
submission_df['VANDVI2'] = submission_df['VANDVI2'].replace([np.inf, -np.inf], np.nan) #handle nan

submission_df['LST_Minus_NDVI'] = (submission_df['lwir11'] / 10) - submission_df['NDVI2']
submission_df['LST_Minus_NDVI'] = submission_df['LST_Minus_NDVI'].replace([np.inf, -np.inf], np.nan)

submission_df['test1']=submission_df['avg_building_height']*(submission_df['lwir11']/10-submission_df['NDVI'])
submission_df['test2']=submission_df['avg_building_height']*(submission_df['NDBI'])
submission_df['test3']=submission_df['building_density']*submission_df['avg_building_height']*(submission_df['lwir11']/10-submission_df['NDVI'])

# Prepare submission features
submission_X = submission_df[features]
submission_X = submission_X.fillna(submission_X.mean())
submission_X_scaled = scaler.transform(submission_X)

# Predict UHI Index
submission_df["UHI Index"] = rf_model.predict(submission_X_scaled)

# Save predictions
submission_df[["Longitude", "Latitude", "UHI Index"]].to_csv("UHI_predictions_buffer.csv", index=False)
print("Predictions saved to UHI_predictions_buffer.csv")

Extracting Buffered Values:   0%|          | 0/1040 [00:00<?, ?it/s]

Extracting Buffered Values: 100%|██████████| 1040/1040 [00:01<00:00, 555.39it/s]


Layers in KML: ['Challenge_footprint']
Predictions saved to UHI_predictions_buffer.csv
